In [1]:
!"C:\Program Files\MongoDB\Server\5.0\bin\mongod.exe" --dbpath="c:\data\db"

{"t":{"$date":"2021-08-18T16:32:13.867+02:00"},"s":"I",  "c":"CONTROL",  "id":23285,   "ctx":"main","msg":"Automatically disabling TLS 1.0, to force-enable TLS 1.0 specify --sslDisabledProtocols 'none'"}
{"t":{"$date":"2021-08-18T16:32:13.869+02:00"},"s":"I",  "c":"NETWORK",  "id":4915701, "ctx":"main","msg":"Initialized wire specification","attr":{"spec":{"incomingExternalClient":{"minWireVersion":0,"maxWireVersion":13},"incomingInternalClient":{"minWireVersion":0,"maxWireVersion":13},"outgoing":{"minWireVersion":0,"maxWireVersion":13},"isInternalClient":true}}}
{"t":{"$date":"2021-08-18T16:32:14.396+02:00"},"s":"W",  "c":"ASIO",     "id":22601,   "ctx":"main","msg":"No TransportLayer configured during NetworkInterface startup"}
{"t":{"$date":"2021-08-18T16:32:14.396+02:00"},"s":"I",  "c":"NETWORK",  "id":4648602, "ctx":"main","msg":"Implicit TCP FastOpen in use."}
{"t":{"$date":"2021-08-18T16:32:14.399+02:00"},"s":"W",  "c":"ASIO",     "id":22601,   "ctx":"main","msg":"No TransportLa

In [2]:
from pymongo import MongoClient

# Connecting to MongoDB

In [3]:
client = MongoClient("mongodb://localhost:27017")

# Helper Functions

In [4]:
def print_cursor(c):
    for i in cursor:
        print(i)

# Dataset 1

In [28]:
db = client.universities

In [29]:
client.drop_database("universities")

In [30]:
db.universities.insert_many([
{
  "country" : "Spain",
  "city" : "Salamanca",
  "name" : "USAL",
  "location" : {
    "type" : "Point",
    "coordinates" : [ -5.6722512,17, 40.9607792 ]
  },
  "students" : [
    { "year" : 2014, "number" : 24774 },
    { "year" : 2015, "number" : 23166 },
    { "year" : 2016, "number" : 21913 },
    { "year" : 2017, "number" : 21715 }
  ]
},
{
  "country" : "Spain",
  "city" : "Salamanca",
  "name" : "UPSA",
  "location" : {
    "type" : "Point",
    "coordinates" : [ -5.6691191,17, 40.9631732 ]
  },
  "students" : [
    { "year" : 2014, "number" : 4788 },
    { "year" : 2015, "number" : 4821 },
    { "year" : 2016, "number" : 6550 },
    { "year" : 2017, "number" : 6125 }
  ]
}
])


db.courses.insert_many([
{
  "university" : "USAL",
  "name" : "Computer Science",
  "level" : "Excellent"
},
{
  "university" : "USAL",
  "name" : "Electronics",
  "level" : "Intermediate"
},
{
  "university" : "USAL",
  "name" : "Communication",
  "level" : "Excellent"
}
])

# Aggregation stages

## MongoDB `$match`

In [31]:
cursor = db.universities.aggregate([
  { "$match" : { "country" : "Spain", "city" : "Salamanca" } }
])

print_cursor(cursor)

{'_id': ObjectId('611d19fc62fa6eb8a867f80b'), 'country': 'Spain', 'city': 'Salamanca', 'name': 'USAL', 'location': {'type': 'Point', 'coordinates': [-5.6722512, 17, 40.9607792]}, 'students': [{'year': 2014, 'number': 24774}, {'year': 2015, 'number': 23166}, {'year': 2016, 'number': 21913}, {'year': 2017, 'number': 21715}]}
{'_id': ObjectId('611d19fc62fa6eb8a867f80c'), 'country': 'Spain', 'city': 'Salamanca', 'name': 'UPSA', 'location': {'type': 'Point', 'coordinates': [-5.6691191, 17, 40.9631732]}, 'students': [{'year': 2014, 'number': 4788}, {'year': 2015, 'number': 4821}, {'year': 2016, 'number': 6550}, {'year': 2017, 'number': 6125}]}


## MongoDB `$project`

In [32]:
cursor = db.universities.aggregate([
  { "$project" : { "_id" : 0, "country" : 1, "city" : 1, "name" : 1 } }
])

print_cursor(cursor)

{'country': 'Spain', 'city': 'Salamanca', 'name': 'USAL'}
{'country': 'Spain', 'city': 'Salamanca', 'name': 'UPSA'}


## MongoDB `$group`

In [33]:
cursor = db.universities.aggregate([
  { "$group" : { "_id" : "$name", "totaldocs" : { "$sum" : 1 } } }
])

print_cursor(cursor)

{'_id': 'USAL', 'totaldocs': 1}
{'_id': 'UPSA', 'totaldocs': 1}


## MongoDB `$out`

In [11]:
db.universities.aggregate([
  { "$group" : { "_id" : "$name", "totaldocs" : { "$sum" : 1 } } },
  { "$out" : "aggResults" }
])

cursor = db.aggResults.find()
print_cursor(cursor)

{'_id': 'USAL', 'totaldocs': 1}
{'_id': 'UPSA', 'totaldocs': 1}


## MongoDB `$unwind`

In [12]:
cursor = db.universities.aggregate([
  { "$match" : { "name" : "USAL" } },
  { "$unwind" : "$students" }
])

print_cursor(cursor)

{'_id': ObjectId('611d19ee62fa6eb8a867f806'), 'country': 'Spain', 'city': 'Salamanca', 'name': 'USAL', 'location': {'type': 'Point', 'coordinates': [-5.6722512, 17, 40.9607792]}, 'students': {'year': 2014, 'number': 24774}}
{'_id': ObjectId('611d19ee62fa6eb8a867f806'), 'country': 'Spain', 'city': 'Salamanca', 'name': 'USAL', 'location': {'type': 'Point', 'coordinates': [-5.6722512, 17, 40.9607792]}, 'students': {'year': 2015, 'number': 23166}}
{'_id': ObjectId('611d19ee62fa6eb8a867f806'), 'country': 'Spain', 'city': 'Salamanca', 'name': 'USAL', 'location': {'type': 'Point', 'coordinates': [-5.6722512, 17, 40.9607792]}, 'students': {'year': 2016, 'number': 21913}}
{'_id': ObjectId('611d19ee62fa6eb8a867f806'), 'country': 'Spain', 'city': 'Salamanca', 'name': 'USAL', 'location': {'type': 'Point', 'coordinates': [-5.6722512, 17, 40.9607792]}, 'students': {'year': 2017, 'number': 21715}}


## MongoDB `$sort`

In [13]:
cursor = db.universities.aggregate([
  { "$match" : { "name" : "USAL" } },
  { "$unwind" : "$students" },
  { "$project" : { "_id" : 0, "students.year" : 1, "students.number" : 1 } },
  { "$sort" : { "students.number" : -1 } }
])

print_cursor(cursor)

{'students': {'year': 2014, 'number': 24774}}
{'students': {'year': 2015, 'number': 23166}}
{'students': {'year': 2016, 'number': 21913}}
{'students': {'year': 2017, 'number': 21715}}


## MongoDB `$limit`

In [14]:
cursor = db.universities.aggregate([
  { "$match" : { "name" : "USAL" } },
  { "$unwind" : "$students" },
  { "$project" : { "_id" : 0, "students.year" : 1, "students.number" : 1 } },
  { "$sort" : { "students.number" : -1 } },
  { "$limit" : 2 }
])

print_cursor(cursor)

{'students': {'year': 2014, 'number': 24774}}
{'students': {'year': 2015, 'number': 23166}}


## MongoDB `$addFields`

In [15]:
cursor = db.universities.aggregate([
  { "$match" : { "name" : "USAL" } },
  { "$addFields" : { "foundation_year" : 1218 } }
])

print_cursor(cursor)

{'_id': ObjectId('611d19ee62fa6eb8a867f806'), 'country': 'Spain', 'city': 'Salamanca', 'name': 'USAL', 'location': {'type': 'Point', 'coordinates': [-5.6722512, 17, 40.9607792]}, 'students': [{'year': 2014, 'number': 24774}, {'year': 2015, 'number': 23166}, {'year': 2016, 'number': 21913}, {'year': 2017, 'number': 21715}], 'foundation_year': 1218}


## MongoDB `$count`

In [16]:
cursor = db.universities.aggregate([
  { "$unwind" : "$students" },
  { "$count" : "total_documents" }
])

print_cursor(cursor)

{'total_documents': 8}


## MongoDB `$lookup`

If you want this query to run fast, you are going to need to index the `name` field in the `universities` collection and the `university` field in the `courses` collection.

In [17]:
cursor = db.universities.aggregate([
  { "$match" : { "name" : "USAL" } },
  { "$project" : { "_id" : 0, "name" : 1 } },
  { "$lookup" : {
      "from" : "courses",
      "localField" : "name",
      "foreignField" : "university",
      "as" : "courses"
  } }
])

print_cursor(cursor)

{'name': 'USAL', 'courses': [{'_id': ObjectId('611d19ee62fa6eb8a867f808'), 'university': 'USAL', 'name': 'Computer Science', 'level': 'Excellent'}, {'_id': ObjectId('611d19ee62fa6eb8a867f809'), 'university': 'USAL', 'name': 'Electronics', 'level': 'Intermediate'}, {'_id': ObjectId('611d19ee62fa6eb8a867f80a'), 'university': 'USAL', 'name': 'Communication', 'level': 'Excellent'}]}


## MongoDB `$sortByCount`

This stage is a shortcut for grouping, counting and then sorting in descending order the number of different values in a field.

In [18]:
cursor = db.courses.aggregate([
  { "$sortByCount" : "$level" }
])

print_cursor(cursor)

{'_id': 'Excellent', 'count': 2}
{'_id': 'Intermediate', 'count': 1}


## MongoDB `$facet`

Sometimes when creating a report on data, you find that you need to do the same preliminary processing for a number of reports, and you are faced with having to create and maintain an intermediate collection.

In [19]:
cursor = db.universities.aggregate([
  { "$match" : { "name" : "USAL" } },
  { "$lookup" : {
      "from" : "courses",
      "localField" : "name",
      "foreignField" : "university",
      "as" : "courses"
  } }
])

print_cursor(cursor)

{'_id': ObjectId('611d19ee62fa6eb8a867f806'), 'country': 'Spain', 'city': 'Salamanca', 'name': 'USAL', 'location': {'type': 'Point', 'coordinates': [-5.6722512, 17, 40.9607792]}, 'students': [{'year': 2014, 'number': 24774}, {'year': 2015, 'number': 23166}, {'year': 2016, 'number': 21913}, {'year': 2017, 'number': 21715}], 'courses': [{'_id': ObjectId('611d19ee62fa6eb8a867f808'), 'university': 'USAL', 'name': 'Computer Science', 'level': 'Excellent'}, {'_id': ObjectId('611d19ee62fa6eb8a867f809'), 'university': 'USAL', 'name': 'Electronics', 'level': 'Intermediate'}, {'_id': ObjectId('611d19ee62fa6eb8a867f80a'), 'university': 'USAL', 'name': 'Communication', 'level': 'Excellent'}]}


In [20]:
cursor = db.universities.aggregate([
  { "$match" : { "name" : "USAL" } },
  { "$lookup" : {
      "from" : "courses",
      "localField" : "name",
      "foreignField" : "university",
      "as" : "courses"
  } },
         { "$unwind" : "$courses" },
         { "$sortByCount" : "$courses.level" }
])

print_cursor(cursor)

{'_id': 'Excellent', 'count': 2}
{'_id': 'Intermediate', 'count': 1}


In [21]:
cursor = db.universities.aggregate([
  { "$match" : { "name" : "USAL" } },
  { "$lookup" : {
      "from" : "courses",
      "localField" : "name",
      "foreignField" : "university",
      "as" : "courses"
  } },
         { "$unwind" : "$students" },
         { "$project" : { "_id" : 0, "students" : 1 } },
         { "$sort" : { "students.number" : 1 } },
         { "$limit" : 1 }
])

print_cursor(cursor)

{'students': {'year': 2017, 'number': 21715}}


In [22]:
cursor = db.universities.aggregate([
  { "$match" : { "name" : "USAL" } },
  { "$lookup" : {
      "from" : "courses",
      "localField" : "name",
      "foreignField" : "university",
      "as" : "courses"
  } },
  { "$facet" : {
      "countingLevels" :
      [
         { "$unwind" : "$courses" },
         { "$sortByCount" : "$courses.level" }
      ],
      "yearWithLessStudents" :
      [
         { "$unwind" : "$students" },
         { "$project" : { "_id" : 0, "students" : 1 } },
         { "$sort" : { "students.number" : 1 } },
         { "$limit" : 1 }
      ]
  } }
])

print_cursor(cursor)

{'countingLevels': [{'_id': 'Excellent', 'count': 2}, {'_id': 'Intermediate', 'count': 1}], 'yearWithLessStudents': [{'students': {'year': 2017, 'number': 21715}}]}


# Dataset 2

Use `mongoimport` to load http://media.mongodb.org/zips.json into your mongod instance.


In [52]:
!"E:\program files\Mongo\mongodb-database-tools-windows-x86_64-100.5.0\bin\mongoimport.exe" --db=local --collection=zipcodes --file="zips.json"

2021-08-18T17:31:44.946+0200	connected to: mongodb://localhost/
2021-08-18T17:31:45.437+0200	29353 document(s) imported successfully. 0 document(s) failed to import.


In [53]:
db = client.local

In [54]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'local')

In [56]:
cursor = db.zipcodes.find({"_id" : "47906"})
print_cursor(cursor)

{'_id': '47906', 'city': 'WEST LAFAYETTE', 'loc': [-86.923661, 40.444025], 'pop': 54702, 'state': 'IN'}


# Aggregation Pipeline

### Return States with Populations above 10 Million

`SELECT state, SUM(pop) AS totalPop
FROM zipcodes
GROUP BY state
HAVING totalPop >= (10*1000*1000)`

In [57]:
cursor = db.zipcodes.aggregate( [
   { "$group": { "_id": "$state", "totalPop": { "$sum": "$pop" } } },
   { "$match": { "totalPop": { "$gte": 10*1000*1000 } } }
] )

print_cursor(cursor)

{'_id': 'PA', 'totalPop': 11881643}
{'_id': 'OH', 'totalPop': 10846517}
{'_id': 'NY', 'totalPop': 17990402}
{'_id': 'CA', 'totalPop': 29754890}
{'_id': 'FL', 'totalPop': 12686644}
{'_id': 'TX', 'totalPop': 16984601}
{'_id': 'IL', 'totalPop': 11427576}


### Return Average City Population by State

In [58]:
cursor = db.zipcodes.aggregate( [
   { "$group": { "_id": { "state": "$state", "city": "$city" }, "pop": { "$sum": "$pop" } } },
   { "$group": { "_id": "$_id.state", "avgCityPop": { "$avg": "$pop" } } }
] )

print_cursor(cursor)

{'_id': 'KS', 'avgCityPop': 3819.884259259259}
{'_id': 'IN', 'avgCityPop': 9271.130434782608}
{'_id': 'WA', 'avgCityPop': 12258.670025188916}
{'_id': 'AK', 'avgCityPop': 2976.4918032786886}
{'_id': 'SD', 'avgCityPop': 1839.6746031746031}
{'_id': 'HI', 'avgCityPop': 15831.842857142858}
{'_id': 'MI', 'avgCityPop': 12087.512353706112}
{'_id': 'DC', 'avgCityPop': 303450.0}
{'_id': 'WI', 'avgCityPop': 7323.00748502994}
{'_id': 'OK', 'avgCityPop': 6155.743639921722}
{'_id': 'NV', 'avgCityPop': 18209.590909090908}
{'_id': 'UT', 'avgCityPop': 9518.508287292818}
{'_id': 'AZ', 'avgCityPop': 20591.16853932584}
{'_id': 'IL', 'avgCityPop': 9954.334494773519}
{'_id': 'MS', 'avgCityPop': 7524.023391812865}
{'_id': 'KY', 'avgCityPop': 4767.164721141375}
{'_id': 'DE', 'avgCityPop': 14481.91304347826}
{'_id': 'VT', 'avgCityPop': 2315.8765432098767}
{'_id': 'WV', 'avgCityPop': 2771.4775888717154}
{'_id': 'TX', 'avgCityPop': 13775.02108678021}
{'_id': 'IA', 'avgCityPop': 3123.0821147356583}
{'_id': 'TN', 

### Return Largest and Smallest Cities by State

In [59]:
cursor = db.zipcodes.aggregate( [
   { "$group":
      {
        "_id": { "state": "$state", "city": "$city" },
        "pop": { "$sum": "$pop" }
      }
   },
   { "$sort": { "pop": 1 } },
   { "$group":
      {
        "_id" : "$_id.state",
        "biggestCity":  { "$last": "$_id.city" },
        "biggestPop":   { "$last": "$pop" },
        "smallestCity": { "$first": "$_id.city" },
        "smallestPop":  { "$first": "$pop" }
      }
   },
  { "$project":
    { "_id": 0,
      "state": "$_id",
      "biggestCity":  { "name": "$biggestCity",  "pop": "$biggestPop" },
      "smallestCity": { "name": "$smallestCity", "pop": "$smallestPop" }
    }
  }
] )

print_cursor(cursor)

{'biggestCity': {'name': 'BILLINGS', 'pop': 78805}, 'smallestCity': {'name': 'MOSBY', 'pop': 7}, 'state': 'MT'}
{'biggestCity': {'name': 'NEWARK', 'pop': 275572}, 'smallestCity': {'name': 'IMLAYSTOWN', 'pop': 17}, 'state': 'NJ'}
{'biggestCity': {'name': 'WASHINGTON', 'pop': 606879}, 'smallestCity': {'name': 'PENTAGON', 'pop': 21}, 'state': 'DC'}
{'biggestCity': {'name': 'BRIDGEPORT', 'pop': 141638}, 'smallestCity': {'name': 'EAST KILLINGLY', 'pop': 25}, 'state': 'CT'}
{'biggestCity': {'name': 'SAINT LOUIS', 'pop': 397802}, 'smallestCity': {'name': 'BENDAVIS', 'pop': 44}, 'state': 'MO'}
{'biggestCity': {'name': 'SIOUX FALLS', 'pop': 102046}, 'smallestCity': {'name': 'ZEONA', 'pop': 8}, 'state': 'SD'}
{'biggestCity': {'name': 'INDIANAPOLIS', 'pop': 348868}, 'smallestCity': {'name': 'WESTPOINT', 'pop': 145}, 'state': 'IN'}
{'biggestCity': {'name': 'DETROIT', 'pop': 963243}, 'smallestCity': {'name': 'LELAND', 'pop': 0}, 'state': 'MI'}
{'biggestCity': {'name': 'TULSA', 'pop': 389072}, 'smal